# Pause, Resume, and Suspend

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [0]:
%matplotlib notebook

from bluesky import RunEngine
from bluesky.plans import scan, count
from bluesky.callbacks import LiveTable, LivePlot

# Set up simulated hardware.
from bluesky.examples import motor, SynGauss
det = SynGauss('det', motor, 'motor', center=0, Imax=1,
               noise='uniform', sigma=1, noise_multiplier=0.1)

RE = RunEngine({})
table = LiveTable(['det', 'motor'])

## Data Acquisition

### Interactively Pausing

In [0]:
# counts forever until interrupted -- press 'stop' button in the notebook (in the terminal, you would use Ctrl+C)
RE(count([det], delay=1, num=None), table)

The RunEngine is paused! Our choices:

|Command    |    Outcome|
|-----|-----|
|`RE.resume()`   |Safely resume plan.|
|`RE.abort()`    |Perform cleanup. Mark as aborted.|
|`RE.stop()`     |Perform cleanup. Mark as success.|
|`RE.halt()`     |Do not perform cleanup --- just stop.|
|`RE.state`      |Check if 'paused' or 'idle'.|
|-----|-----|

While the RunEngine is paused, you have control of the prompt. You can check on things.

In [0]:
1 + 1

In [0]:
det.read()

### Safely Resuming

In [0]:
# Again, we need to press 'stop' or this will run forever....
RE.resume()

### Exit options: abort, stop, and halt

Paused again. Once more, our choices:

|Command    |    Outcome|
|-----|-----|
|`RE.resume()`   |Safely resume plan.|
|`RE.abort()`    |Perform cleanup. Mark as aborted.|
|`RE.stop()`     |Perform cleanup. Mark as success.|
|`RE.halt()`     |Do not perform cleanup --- just stop.|
|`RE.state`      |Check if 'paused' or 'idle'.|
|-----|-----|

In [0]:
RE.abort()

Compare 'abort' to 'stop' and 'halt':

In [0]:
RE(count([det], delay=1, num=None), table)

In [0]:
RE.stop()

In [0]:
RE(count([det], delay=1, num=None), table)

In [0]:
RE.halt()

### What happens if you try to run a new plan from a paused state?

In [0]:
RE(count([det], delay=1, num=None), table)

In [0]:
RE(count([det], delay=1, num=None), table)

In [0]:
RE.state

In [0]:
RE.abort()

In [0]:
RE.state

### Configure a "suspender"

Suspenders are agents set up in advance to run in the background and pause/resume a scan in response to some condition (like a beam dump).

In [0]:
from bluesky.suspenders import SuspendFloor
import ophyd

current = ophyd.Signal(name='beam_current')

sus = SuspendFloor(current, 50, sleep=1)
RE.install_suspender(sus)

In [0]:
##### Simulate a beam dump and recovery after 3 seconds....
import threading
import time

def delay_dump_delay_recover():
    time.sleep(3)
    sig.set(0)
    time.sleep(3)
    sig.set(100)

sig.set(100)
thread = threading.Thread(target=delay_dump_delay_recover)
thread.start()
#####

RE(count([det], num=10, delay=1), table)